In [1]:
import timeit
start = timeit.default_timer()

import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'
import sys
print(sys.version)

3.10.9 (main, Mar  1 2023, 18:23:06) [GCC 11.2.0]


In [2]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
import numpy as np
import pyarrow as pa
import pyspark as ps
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ljendrusch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
sp = [spark.read.text('hdfs://orion11:14001/books/*{:d}.*'.format(i), wholetext=True) for i in range(10)]

In [4]:
stop_words = set(stopwords.words('english'))

def filter_stops(text):
    return ','.join([w for w in text if w not in stop_words])

In [6]:
for i in range(10):
    sp[i].rdd \
        .map(lambda x: x[0].split('\n', 2)) \
        .map(lambda x: ( x[0][7:], x[1][8:], re.sub(r'\s+',' ', re.sub(r'[^\w\s]','',x[2])).lower().split() )) \
        .map(lambda x: ( x[0], x[1], filter_stops(x[2]) )) \
        .toDF(['title','author','text']).write.csv('hdfs://orion11:14001/cleaned_books/{:d}'.format(i))

In [7]:
print(timeit.default_timer()-start)

128.88671489502303
